# Developing a Coordinate Transformation Routine for Solving Hamilton's EOM in Spherical Coordinates

### Author: Siddharth Mahesh

## Step : Import Modules




In [1]:
import numpy as np
import sympy as sp


## Step : Transforming Cartesian Coordinates to Spherical Polar

We define the transformation from cartesian coordinates $(x,y,z)$ to spherical polar $(r,\phi,\theta)$ as follows:

$$
r = \sqrt{x^2 + y^2 + z^2}\\
\phi = \tan^{-1}\left(\frac{y}{x}\right)\\
\theta = \cos^{-1}\left(\frac{z}{r}\right)
$$

The momenta are then given by:

$$
p^r = p^x\cos\phi\sin\theta + p^y\sin\phi\sin\theta + p^z\cos\theta \\
p^\phi = -p^xr\sin\phi\cos\theta + p^yr\cos\phi\sin\theta\\
p^\theta = p^xr\cos\phi\cos\theta + p^yr\sin\phi\cos\theta - p^z\sin\theta
$$

In [ ]:
def TransformCartesianToSpherical(values,off,sep):
    x , y , z = values[0] , values[1] , values[2]
    px , py , pz = values[off+sep*0] , values[off+sep*1] , values[off+sep*2]
    r = np.sqrt(x*x + y*y + z*z)
    phi = np.arctan2(y,x)
    theta = np.arccos(z,r)
    c_phi , s_phi = np.cos(phi) , np.sin(phi)
    c_theta , s_theta = np.cos(theta) , np.sin(theta)
    sph_values = np.zeros(len(values))
    sph_values[0] = r
    sph_values[1] = phi
    sph_values[2] = theta
    sph_values[off+sep*0] = px*c_phi*s_theta + py*s_phi*s_theta + pz*c_theta
    sph_values[off+sep*1] = -px*r*s_phi*s_theta + py*c_phi*s_theta
    sph_values[off+sep*2] = px*r*c_phi*c_theta + py*r*s_phi*c_theta - pz*r*s_theta
    return sph_values   

## Step : Transforming Spherical Polar Coordinates to Cartesian

We define the transformation from spherical coordinates to cartesian as follows:

$$
x = r\cos\phi\sin\theta\\
y = r\sin\phi\sin\theta\\
z = r\cos\theta
$$

The momenta then transform as:

$$
p^x = p^r\cos\phi\sin\theta - \frac{p^\phi\sin\phi\sin\theta}{r} + \frac{p^\theta\cos\phi\cos\theta}{r}\\
p^y = p^r\sin\phi\sin\theta + \frac{p^\phi\cos\phi\sin\theta}{r} + \frac{p^\theta\sin\phi\cos\theta}{r}\\
p^z = p^r\cos\theta - \frac{p^\theta\sin\theta}{r}
$$

In [ ]:
def TransformSphericalToCartesian(values,off,sep):
    r , phi , theta = values[0] , values[1] , values[2]
    pr , pphi , ptheta = values[off+sep*0] , values[off+sep*1] , values[off+sep*2]
    c_phi , s_phi = np.cos(phi) , np.sin(phi)
    c_theta , s_theta = np.cos(theta) , np.sin(theta)
    cart_values = np.zeros(len(values))
    cart_values[0] = r*c_phi*s_theta
    cart_values[1] = r*s_phi*s_theta
    cart_values[2] = r*c_theta
    cart_values[off+sep*0] = pr*c_phi*s_theta + (-pphi*s_phi*s_theta + ptheta*c_phi*c_theta)/r
    cart_values[off+sep*1] = pr*s_phi*s_theta + (pphi*c_phi*s_theta + ptheta*s_phi*c_theta)/r
    cart_values[off+sep*2] = pr*c_theta - ptheta*s_theta/r
    return sph_values

## Step : The Jacobian Matrix of Transformation

In order to translate partial derivatives of the Hamiltonian from cartesian coordinates to spherical polar, we must provide a dictionary of how partial derivatives transform from one coordinate system to the other. This is best encoded in the Jacobian matrix. If we were to define the cartesian phase space as $\eta^C = (x,y,z,p^x,p^y,p^z)$ and likewise define the spherical polar phase space as $\eta^S$, then the Jacobian can be defined as:

$$
\mathbf{J} = \frac{\partial\eta^S}{\partial\eta^C}\\
J_{ij} = \frac{\partial\eta^S_i}{\partial\eta^C_j}
$$

This can be given in cartesian coordinates as:

$$
\mathbf{J} = \begin{pmatrix}
\frac{x}{\sqrt{x^2+y^2+z^2}} & \frac{y}{\sqrt{x^2+y^2+z^2}} &
   \frac{z}{\sqrt{x^2+y^2+z^2}} & 0 & 0 & 0 \\
 \frac{y}{x^2+y^2} & -\frac{x}{x^2+y^2} & 0 & 0 & 0 & 0 \\
 -\frac{x z}{\sqrt{x^2+y^2} \left(x^2+y^2+z^2\right)} & -\frac{y
   z}{\sqrt{x^2+y^2} \left(x^2+y^2+z^2\right)} &
   \frac{\sqrt{x^2+y^2}}{x^2+y^2+z^2} & 0 & 0 & 0 \\
 \frac{p^x}{\sqrt{x^2+y^2+z^2}}-\frac{x (p^x x+p^y
   y+p^z z)}{\left(x^2+y^2+z^2\right)^{3/2}} &
   \frac{p^y}{\sqrt{x^2+y^2+z^2}}-\frac{y (p^x x+p^y
   y+p^z z)}{\left(x^2+y^2+z^2\right)^{3/2}} &
   \frac{p^z}{\sqrt{x^2+y^2+z^2}}-\frac{z (p^x x+p^y
   y+p^z z)}{\left(x^2+y^2+z^2\right)^{3/2}} &
   \frac{x}{\sqrt{x^2+y^2+z^2}} & \frac{y}{\sqrt{x^2+y^2+z^2}} &
   \frac{z}{\sqrt{x^2+y^2+z^2}} \\
 p^y & -p^x & 0 & -y & x & 0 \\
 -\frac{(p^x x z+p^y y z)
   x}{\left(x^2+y^2\right)^{3/2}}-\frac{p^z
   x}{\sqrt{x^2+y^2}}+\frac{p^x z}{\sqrt{x^2+y^2}} &
   -\frac{(p^x x z+p^y y z)
   y}{\left(x^2+y^2\right)^{3/2}}-\frac{p^z
   y}{\sqrt{x^2+y^2}}+\frac{p^y z}{\sqrt{x^2+y^2}} & \frac{p^x
   x+p^y y}{\sqrt{x^2+y^2}} & \frac{x z}{\sqrt{x^2+y^2}} & \frac{y
   z}{\sqrt{x^2+y^2}} & -\sqrt{x^2+y^2} \\
\end{pmatrix}
$$

But since we will have in our possession spherical polar coordinates, alternatively we can rewrite the Jacobian as:

$$
\mathbf{J} = \begin{pmatrix}
 \cos \phi \sin \theta & \sin \theta \sin \phi & \cos \theta
  & 0 & 0 & 0 \\
 \frac{\csc \theta \sin \phi}{r} & -\frac{\cos \phi \csc \theta
 }{r} & 0 & 0 & 0 & 0 \\
 -\frac{\cos \theta \cos \phi}{r} & -\frac{\cos \theta \sin \phi
 }{r} & \frac{\sin \theta}{r} & 0 & 0 & 0 \\
 \frac{p^\theta \cos \theta \cos \phi-p^\phi \sin
   \theta \sin \phi}{r^2} & \frac{p^\phi \cos \phi \sin
   \theta+p^\theta \cos \theta \sin \phi}{r^2} &
   -\frac{p^\theta \sin \theta}{r^2} & \cos \phi \sin
   \theta & \sin \theta \sin \phi & \cos \theta \\
 \frac{p^\phi \cos \phi \sin \theta+p^r r \sin \phi
  \sin \theta+p^\theta \cos \theta \sin \phi}{r} &
   \frac{p^\phi \sin \theta \sin \phi-\cos \phi
   (p^\theta \cos \theta+p^r r \sin \theta)}{r} & 0 &
   -r \sin \theta \sin \phi & r \cos \phi \sin \theta & 0 \\
 \frac{p^\theta \cos \phi \sin \theta-\cos \theta
   (p^r r \cos \phi+p^\phi \sin \phi)}{r} &
   \frac{p^\phi \cos \theta \cos \phi-p^r r \cos
   \theta \sin \phi+p^\theta \sin \theta \sin \phi
 }{r} & \frac{p^\theta \cos \theta}{r}+p^r \sin
   \theta & r \cos \theta \cos \phi & r \cos \theta \sin \phi
  & -r \sin \theta \\
\end{pmatrix}
$$

In [3]:
def JacobianSphericalbyCart(values):
    r , phi , theta = values[0] , values[1] , values[2]
    pr , pphi , ptheta = values[off+sep*0] , values[off+sep*1] , values[off+sep*2]
    c_phi , s_phi = np.cos(phi) , np.sin(phi)
    c_theta , s_theta = np.cos(theta) , np.sin(theta)
    rinv = 1/r
    csc_theta = 1/s_theta
    jacobiansphericalbycart = np.array([[c_phi*s_theta, s_theta*s_phi, c_theta,0, 0, 0], [(csc_theta*s_phi)*rinv, -((c_phi*csc_theta)*rinv), 0, 0, 0,0], [-((c_theta*c_phi)*rinv), -((c_theta*s_phi)*rinv), s_theta*rinv, 0, 0, 0], [(ptheta*c_theta*c_phi - pphi*s_theta*s_phi)*(rinv*rinv), (pphi*c_phi*s_theta + ptheta*c_theta*s_phi)*(rinv*rinv), -((ptheta*s_theta)*(rinv*rinv)), c_phi*s_theta, s_theta*s_phi, c_theta], [(pphi*c_phi*s_theta + ptheta*c_theta*s_phi + pr*r*s_theta*s_phi)*rinv, (-c_phi*(ptheta*c_theta + pr*r*s_theta) + pphi*s_theta*s_phi)*rinv,0, -r*s_theta*s_phi, r*c_phi*s_theta, 0], [(ptheta*c_phi*s_theta - c_theta*(pr*r*c_phi + pphi*s_phi))*rinv, (pphi*c_theta*c_phi - pr*r*c_theta*s_phi + ptheta*s_theta*s_phi)*rinv, (ptheta*c_theta)*rinv + pr*s_theta, r*c_theta*c_phi, r*c_theta*s_phi, -r*s_theta]])
    return jacobiansphericalbycart

## Step : Hamilton's Equations in Symplectic Form

Consider an abstract Hamiltonian $\mathcal{H}$ that is given in cartesian coordinates. The propagation of the dynamical variables $\eta = \left(x,y,z,p^x,p^y,p^z\right)$ over time can be written in *symplectic form* as follows:

$$
\dot{\eta} = \mathbf{\bar{I}}\frac{\partial\mathcal{H}}{\partial\eta}
$$

Where $\mathbf{\bar{I}}$ is a matrix that, for $N$ spatial variables, can be given in block form as:

$$
\mathbf{\bar{I}} = \begin{pmatrix}
\mathbf{0}_{N} & \mathbf{I}_{N}\\
-\mathbf{I}_{N} & \mathbf{0}_{N}
\end{pmatrix}
$$

Where $\mathbf{0}_{N}$ represents the $N\times N$ zero matrix while $\mathbf{I}_{N}$ is the $N\times N$ identity matrix. This matrix will prove valuable in generating the left-hand-sides for the time evolution of spherical polar coordinates and thus, needs to be implemented.

Note: While the term symplectic generally refers to Hamiltonians that conserve energy, the name symplectic form here is used to represent writing the Hamilton's Equations in the above form.

In [2]:
def ConstructIbar():
    return np.block([[np.zeros([3,3]) , np.identity(3)],[-np.identity(3) , np.zeros([3,3])]])

[[ 0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [-1. -0. -0.  0.  0.  0.]
 [-0. -1. -0.  0.  0.  0.]
 [-0. -0. -1.  0.  0.  0.]]


## Step : The EOM for Spherical Polar Coordinates in Symplectic Form

The Equations of Motion satisfied by the spherical polar variables $\eta^S$ are given by:

$$
\dot{\eta}^S = \frac{\partial\eta^S}{\partial\eta}\dot{\eta}
$$

Where, $\eta$ are the cartesian variables. Thus,

$$
\dot{\eta}^S = \mathbf{J}\mathbf{\bar{I}}\frac{\partial\mathcal{H}}{\partial\eta}
$$